In [1]:
from transformers import DistilBertTokenizerFast, AutoModelForQuestionAnswering, DefaultDataCollator, Trainer, TrainingArguments, set_seed
from datasets import *
import numpy as np
import torch
import ast

In [2]:
SEED = 42
set_seed(SEED)

# any combination of these years and dataset types can be used
year = 2020
# year = 2022
dataset_type = "full"
# dataset_type = "smaller"

# only combination of these years and dataset types can be used
# year = 2022
# dataset_type = "handwritten"

local_models_path = '../../data/models'

model_name = 'distilbert-base-cased-distilled-squad'

### Load tokenizer and model

In [3]:
tokenizer = tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = model = AutoModelForQuestionAnswering.from_pretrained(model_name)

### Load the dataset

In [4]:
# Load the dataset from file and split it into train and test datasets
if dataset_type == "full":
    data = load_dataset('csv', data_files=f"../../data/clean/sustainability-report-{year}-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)
elif dataset_type == "smaller":
    data = load_dataset('csv', data_files=f"../../data/clean/sustainability-report-{year}-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)
    data["train"] = data["train"].select(range(len(data["train"]) // 2))
elif dataset_type == "handwritten":
    data = load_dataset('csv', data_files=f"../../data/clean/sr-2022-questions-answers-ALL-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)
else:
    raise Exception("Invalid dataset type")

Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-d8382661cd597e83/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-c261d5613d28d856.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-e61829c1e4a24b65.arrow


In [5]:
# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data["test"] = data["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data["test"] = data["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
data["test"] = data["test"].remove_columns(["text", "answer_start"])

data["train"] = data["train"].map(
    lambda example: ast.literal_eval(example["answers"]))
data["train"] = data["train"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                  "text": example["text"], "answer_start": example["answer_start"]}})
data["train"] = data["train"].remove_columns(["text", "answer_start"])

Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0b15501cefb41ff7.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-e4de42d02343959f.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-65eb14b3b79cbed9.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-01ba1a0d303da5fc.arrow


In [6]:
def tokenize_sample_data(data):
    # Tokenize the data
    tokenized_feature = tokenizer(
        data["question"],
        data["context"],
        max_length=384,
        return_overflowing_tokens=True,
        stride=128,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
    )

    # When it overflows, multiple rows will be returned for a single example.
    # The following then gets the array of corresponding the original sample index.
    sample_mapping = tokenized_feature.pop("overflow_to_sample_mapping")
    # Get the array of [start_char, end_char + 1] in each token.
    # The shape is [returned_row_size, max_length]
    offset_mapping = tokenized_feature.pop("offset_mapping")

    start_positions = []
    end_positions = []
    for i, offset in enumerate(offset_mapping):
        sample_index = sample_mapping[i]
        answers = data["answers"][sample_index]
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0]) - 1
        # The format of sequence_ids is [None, 0, ..., 0, None, None, 1, ..., 1, None, None, ...]
        # in which question's token is 0 and contex's token is 1
        sequence_ids = tokenized_feature.sequence_ids(i)
        # find the start and end index of context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
        # Set start positions and end positions in inputs_ids
        # Note: The second element in offset is end_char + 1
        # if offset[context_start][0] > end_char or offset[context_end][1] <= start_char:
        if not (offset[context_start][0] <= start_char and end_char < offset[context_end][1]):
            # The case that answer is not inside the context
            # Note : Some tokenizer (such as, tokenizer in rinna model) doesn't place CLS
            # for the first token in sequence, and I then set -1 as positions.
            # (Later I'll process rows with start_positions=-1.)
            start_positions.append(-1)
            end_positions.append(-1)
        else:
            # The case that answer is found in the context

            # Set start position
            idx = context_start
            while offset[idx][0] < start_char:
                idx += 1
            if offset[idx][0] == start_char:
                start_positions.append(idx)
            else:
                start_positions.append(idx - 1)

            # Set end position
            idx = context_end
            while offset[idx][1] > end_char + 1:
                idx -= 1
            if offset[idx][1] == end_char + 1:
                end_positions.append(idx)
            else:
                end_positions.append(idx + 1)

    # Build result
    tokenized_feature["start_positions"] = start_positions
    tokenized_feature["end_positions"] = end_positions
    return tokenized_feature


# Run conversion
tokenized_ds = data.map(
    tokenize_sample_data,
    remove_columns=["id", "context", "question", "answers"],
    batched=True,
    batch_size=128)

Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-9a33255025e48b76.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-f93b11b1dbea438d.arrow


In [7]:
name = model_name.split("/")[-1]
output_dir = f"{local_models_path}/{name}-finetuned-NLB-QA-{year}-{dataset_type}"

# Set the data collator
data_collator = DefaultDataCollator()

# Define the training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=25,
    fp16=True,
    push_to_hub=False,
    load_best_model_at_end=True
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
)

In [8]:
trainer.train()
trainer.save_model(output_dir)

c:\Users\Luka\miniconda3\envs\project_ds\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.6663624048233032, 'eval_runtime': 0.1613, 'eval_samples_per_second': 347.269, 'eval_steps_per_second': 24.805, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

...

{'eval_loss': 0.7831096649169922, 'eval_runtime': 0.1661, 'eval_samples_per_second': 337.094, 'eval_steps_per_second': 24.078, 'epoch': 23.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.7843310236930847, 'eval_runtime': 0.1652, 'eval_samples_per_second': 339.044, 'eval_steps_per_second': 24.217, 'epoch': 24.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.7848501205444336, 'eval_runtime': 0.166, 'eval_samples_per_second': 337.354, 'eval_steps_per_second': 24.097, 'epoch': 25.0}
{'train_runtime': 85.6093, 'train_samples_per_second': 37.671, 'train_steps_per_second': 2.628, 'train_loss': 0.06661239200168186, 'epoch': 25.0}
